In [1]:
import cv2
import numpy as np
import svgwrite

In [6]:
def draw_circle(dwg, center, radius, color='blue'):
    dwg.add(dwg.circle(center=center, r=radius, fill='none', stroke=color, stroke_width=1))

In [9]:
def generate_svg_from_corners(corners, output_filename="detected_box.svg", img_width=None, img_height=None):
    if corners is None or len(corners) == 0:
        print("未检测到角点。")
        return

    if img_width is None or img_height is None:
        # 简单估计画布大小，可能需要根据实际情况调整
        min_x = min(c[0][0] for c in corners)
        max_x = max(c[0][0] for c in corners)
        min_y = min(c[0][1] for c in corners)
        max_y = max(c[0][1] for c in corners)
        width = max_x - min_x + 20
        height = max_y - min_y + 20
        translate_x = -min_x + 10
        translate_y = -min_y + 10
    else:
        width = img_width
        height = img_height
        translate_x = 0
        translate_y = 0

    dwg = svgwrite.Drawing(output_filename, width=width, height=height)
    # group = dwg.g(transform=f"translate({translate_x}, {translate_y})") # 不需要将圆形添加到 group

    for corner in corners:
        x, y = corner.ravel()
        draw_circle(dwg, (x + translate_x, y + translate_y), 5, 'red') # 直接添加到 dwg，并考虑平移

    # dwg.add(group) # 不需要添加 group
    dwg.save()
    print(f"检测到的角点已保存到 {output_filename}")

In [11]:
if True:
    image_path = "package.jpeg"  # 将 "your_image.png" 替换为你的刀板图文件路径
    img = cv2.imread(image_path)
    if img is None:
        print(f"无法加载图像: {image_path}")
    else:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray = np.float32(gray)

        # Shi-Tomasi 角点检测参数
        max_corners = 100  # 你希望检测到的最大角点数
        quality_level = 0.01
        min_distance = 10

        corners = cv2.goodFeaturesToTrack(gray, maxCorners=max_corners, qualityLevel=quality_level, minDistance=min_distance)

        if corners is not None:
            print(f"检测到 {len(corners)} 个角点。")
            generate_svg_from_corners(corners, img_width=img.shape[1], img_height=img.shape[0])
        else:
            print("未检测到角点。请尝试调整检测参数。")

        # 可选: 在原始图像上绘制检测到的角点并显示
        img_with_corners = img.copy()
        if corners is not None:
            for corner in corners:
                x, y = corner.ravel()
                cv2.circle(img_with_corners, (int(x), int(y)), 5, (0, 0, 255), -1)
            cv2.imshow("Detected Corners", img_with_corners)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

检测到 70 个角点。


TypeError: '195.0' is not a valid value for attribute 'cx' at svg-element <circle>.

In [ ]:
import cv2
import numpy as np
import svgwrite

def draw_circle(dwg, center_x, center_y, radius, color='blue'):
    # 确保坐标是 Python 的数字类型
    dwg.add(dwg.circle(center=(float(center_x), float(center_y)), r=float(radius), fill='none', stroke=color, stroke_width=1))

def generate_svg_from_corners(corners, output_filename="detected_box.svg", img_width=None, img_height=None):
    if corners is None or len(corners) == 0:
        print("未检测到角点。")
        return

    if img_width is None or img_height is None:
        min_x = min(c[0][0] for c in corners)
        max_x = max(c[0][0] for c in corners)
        min_y = min(c[0][1] for c in corners)
        max_y = max(c[0][1] for c in corners)
        # 确保画布尺寸是正数
        canvas_width = max(1, max_x - min_x + 20)
        canvas_height = max(1, max_y - min_y + 20)
        translate_x = -min_x + 10
        translate_y = -min_y + 10
    else:
        canvas_width = img_width
        canvas_height = img_height
        translate_x = 0
        translate_y = 0

    dwg = svgwrite.Drawing(output_filename, size=(canvas_width, canvas_height)) # 使用 size 参数

    for corner in corners:
        x, y = corner.ravel()
        # 将 NumPy float 转换为 Python float (或 int)
        draw_circle(dwg, float(x + translate_x), float(y + translate_y), 5, 'red')

    dwg.save()
    print(f"检测到的角点已保存到 {output_filename}")

if True:
    image_path = "package.jpeg"  # 替换为您的文件路径
    img = cv2.imread(image_path)
    if img is None:
        print(f"无法加载图像: {image_path}")
    else:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        max_corners = 200
        quality_level = 0.01
        min_distance = 10
        block_size = 7

        corners = cv2.goodFeaturesToTrack(gray,
                                          maxCorners=max_corners,
                                          qualityLevel=quality_level,
                                          minDistance=min_distance,
                                          blockSize=block_size)

        if corners is not None:
            print(f"检测到 {len(corners)} 个角点。")
            generate_svg_from_corners(corners, output_filename="detected_box_corners.svg", img_width=img.shape[1], img_height=img.shape[0])

            # 在原始图像上绘制角点
            img_with_corners = img.copy()
            for corner in corners:
                x, y = corner.ravel()
                cv2.circle(img_with_corners, (int(x), int(y)), 5, (0, 0, 255), -1)

            # 显示带有角点的图像
            cv2.imshow('Detected Corners on Original Image', img_with_corners)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

            # 保存带有角点的图像
            output_image_path = "detected_box_on_image.png"
            cv2.imwrite(output_image_path, img_with_corners)
            print(f"带有角点的图像已保存为 {output_image_path}")

        else:
            print("未检测到角点。请尝试调整检测参数。")

In [14]:
def create_parametric_box(output_filename="parametric_box.svg",
                           box_length=200,
                           box_width=80,
                           box_height=60,
                           flap_top_length=70,
                           flap_side_length=40,
                           tab_width=15,
                           tab_height=19,
                           notch_width=3.5,
                           notch_height=11.5,
                           corner_radius=2):
    """
    根据参数创建参数化的包装盒刀板图 SVG 文件。
    """
    dwg = svgwrite.Drawing(output_filename, profile='tiny')

    # --- 计算关键坐标和尺寸 ---

    # 主体矩形
    main_rect_width = 2 * box_length + 2 * box_height
    main_rect_height = box_width
    main_rect_origin = (0, flap_top_length)

    # 顶部盖片
    top_flap_origin = (0, 0)
    top_flap_width = main_rect_width
    top_flap_height = flap_top_length

    # 底部盖片
    bottom_flap_origin = (0, main_rect_origin[1] + main_rect_height)
    bottom_flap_width = main_rect_width
    bottom_flap_height = flap_top_length

    # 左侧盖片
    left_flap_origin = (flap_top_length, main_rect_origin[1])
    left_flap_width = box_height
    left_flap_height = box_width

    # 右侧盖片
    right_flap_origin = (flap_top_length + box_length + box_height, main_rect_origin[1])
    right_flap_width = box_height
    right_flap_height = box_width

    # 左侧插舌
    left_tab_origin_top = (left_flap_origin[0] - tab_height, left_flap_origin[1] + 1.5) # 根据图纸估算
    left_tab_origin_bottom = (left_flap_origin[0] - tab_height, left_flap_origin[1] + box_width - 1.5 - tab_height) # 根据图纸估算

    # 右侧插舌
    right_tab_origin_top = (right_flap_origin[0] + right_flap_width + tab_height, right_flap_origin[1] + 1.5) # 根据图纸估算
    right_tab_origin_bottom = (right_flap_origin[0] + right_flap_width + tab_height, right_flap_origin[1] + box_width - 1.5 - tab_height) # 根据图纸估算

    # 左侧切口
    left_notch_origin_top = (flap_top_length + 1.5, main_rect_origin[1] + (box_width - 2 * notch_height) / 3) # 根据图纸估算
    left_notch_origin_bottom = (flap_top_length + 1.5, main_rect_origin[1] + 2 * (box_width - 2 * notch_height) / 3 + notch_height) # 根据图纸估算

    # 右侧切口
    right_notch_origin_top = (flap_top_length + box_length + box_height - 1.5 - notch_width, main_rect_origin[1] + (box_width - 2 * notch_height) / 3) # 根据图纸估算
    right_notch_origin_bottom = (flap_top_length + box_length + box_height - 1.5 - notch_width, main_rect_origin[1] + 2 * (box_width - 2 * notch_height) / 3 + notch_height) # 根据图纸估算

    # --- 绘制形状 ---

    # 主体矩形
    dwg.add(dwg.rect(main_rect_origin, (main_rect_width, main_rect_height), fill='none', stroke='black'))

    # 顶部盖片 (需要绘制圆角)
    top_path = dwg.path(fill='none', stroke='black')
    top_path.push(f"M {top_flap_origin[0]}, {top_flap_origin[1] + corner_radius}")
    top_path.push(f"a {corner_radius},{corner_radius} 0 0 1 {corner_radius}, -{corner_radius}")
    top_path.push(f"H {top_flap_width - corner_radius}")
    top_path.push(f"a {corner_radius},{corner_radius} 0 0 1 {corner_radius}, {corner_radius}")
    top_path.push(f"V {top_flap_height}")
    top_path.push(f"H {top_flap_origin[0]}")
    top_path.push("Z")
    dwg.add(top_path)

    # 底部盖片 (需要绘制圆角)
    bottom_path = dwg.path(fill='none', stroke='black')
    bottom_path.push(f"M {bottom_flap_origin[0]}, {bottom_flap_origin[1] + bottom_flap_height - corner_radius}")
    bottom_path.push(f"a {corner_radius},{corner_radius} 0 0 1 {corner_radius}, {corner_radius}")
    bottom_path.push(f"H {bottom_flap_width - corner_radius}")
    bottom_path.push(f"a {corner_radius},{corner_radius} 0 0 1 {corner_radius}, -{corner_radius}")
    bottom_path.push(f"V {bottom_flap_origin[1]}")
    bottom_path.push(f"H {bottom_flap_origin[0]}")
    bottom_path.push("Z")
    dwg.add(bottom_path)

    # 侧边盖片
    dwg.add(dwg.rect(left_flap_origin, (left_flap_width, left_flap_height), fill='none', stroke='black'))
    dwg.add(dwg.rect(right_flap_origin, (right_flap_width, right_flap_height), fill='none', stroke='black'))

    # 插舌
    dwg.add(dwg.rect(left_tab_origin_top, (tab_height, tab_width), fill='none', stroke='black'))
    dwg.add(dwg.rect(left_tab_origin_bottom, (tab_height, tab_width), fill='none', stroke='black'))
    dwg.add(dwg.rect(right_tab_origin_top, (tab_height, tab_width), fill='none', stroke='black'))
    dwg.add(dwg.rect(right_tab_origin_bottom, (tab_height, tab_width), fill='none', stroke='black'))

    # 切口
    dwg.add(dwg.rect(left_notch_origin_top, (notch_width, notch_height), fill='none', stroke='black'))
    dwg.add(dwg.rect(left_notch_origin_bottom, (notch_width, notch_height), fill='none', stroke='black'))
    dwg.add(dwg.rect(right_notch_origin_top, (notch_width, notch_height), fill='none', stroke='black'))
    dwg.add(dwg.rect(right_notch_origin_bottom, (notch_width, notch_height), fill='none', stroke='black'))

    # 折叠线 (虚线)
    fold_lines = [
        (main_rect_origin[0], main_rect_origin[1]), (main_rect_origin[0] + box_height, main_rect_origin[1]), # 左侧折叠
        (main_rect_origin[0] + box_height + box_length, main_rect_origin[1]), (main_rect_origin[0] + 2 * box_height + box_length, main_rect_origin[1]), # 中间折叠
        (main_rect_origin[0] + 2 * box_height + 2 * box_length, main_rect_origin[1]), (main_rect_origin[0] + 2 * box_height + 2 * box_length + box_height, main_rect_origin[1]), # 右侧折叠

        (main_rect_origin[0], main_rect_origin[1] + box_width), (main_rect_origin[0] + box_height, main_rect_origin[1] + box_width), # 左侧折叠
        (main_rect_origin[0] + box_height + box_length, main_rect_origin[1] + box_width), (main_rect_origin[0] + 2 * box_height + box_length, main_rect_origin[1] + box_width), # 中间折叠
        (main_rect_origin[0] + 2 * box_height + 2 * box_length, main_rect_origin[1] + box_width), (main_rect_origin[0] + 2 * box_height + 2 * box_length + box_height, main_rect_origin[1] + box_width), # 右侧折叠

        (top_flap_origin[0], top_flap_origin[1] + flap_top_length), (main_rect_origin[0], main_rect_origin[1]), # 顶部盖片折叠
        (top_flap_origin[0] + box_height, top_flap_origin[1] + flap_top_length), (main_rect_origin[0] + box_height, main_rect_origin[1]), # 顶部盖片折叠
        (top_flap_origin[0] + box_height + box_length, top_flap_origin[1] + flap_top_length), (main_rect_origin[0] + box_height + box_length, main_rect_origin[1]), # 顶部盖片折叠
        (top_flap_origin[0] + 2 * box_height + box_length, top_flap_origin[1] + flap_top_length), (main_rect_origin[0] + 2 * box_height + box_length, main_rect_origin[1]), # 顶部盖片折叠

        (bottom_flap_origin[0], bottom_flap_origin[1]), (main_rect_origin[0], main_rect_origin[1] + box_width), # 底部盖片折叠
        (bottom_flap_origin[0] + box_height, bottom_flap_origin[1]), (main_rect_origin[0] + box_height, main_rect_origin[1] + box_width), # 底部盖片折叠
        (bottom_flap_origin[0] + box_height + box_length, bottom_flap_origin[1]), (main_rect_origin[0] + box_height + box_length, main_rect_origin[1] + box_width), # 底部盖片折叠
        (bottom_flap_origin[0] + 2 * box_height + box_length, bottom_flap_origin[1]), (main_rect_origin[0] + 2 * box_height + box_length, main_rect_origin[1] + box_width), # 底部盖片折叠

        (left_flap_origin[0], left_flap_origin[1]), (main_rect_origin[0], main_rect_origin[1]), # 左侧盖片折叠
        (left_flap_origin[0], left_flap_origin[1] + box_width), (main_rect_origin[0] + box_height, main_rect_origin[1] + box_width), # 左侧盖片折叠

        (right_flap_origin[0], right_flap_origin[1]), (main_rect_origin[0] + 2 * box_height + 2 * box_length, main_rect_origin[1]), # 右侧盖片折叠
        (right_flap_origin[0], right_flap_origin[1] + box_width), (main_rect_origin[0] + 2 * box_height + 2 * box_length + box_height, main_rect_origin[1] + box_width), # 右侧盖片折叠
    ]
    for i in range(0, len(fold_lines), 2):
        start = fold_lines[i]
        end = fold_lines[i+1]
        dwg.add(dwg.line(start, end, stroke='gray', stroke_dasharray="5,5"))

    dwg.save()
    print(f"参数化包装盒刀板图已保存为 {output_filename}")

In [15]:
create_parametric_box(
        output_filename="my_parametric_box.svg",
        box_length=200,
        box_width=80,
        box_height=60,
        flap_top_length=70,
        flap_side_length=40,
        tab_width=15,
        tab_height=19,
        notch_width=3.5,
        notch_height=11.5,
        corner_radius=2
    )

参数化包装盒刀板图已保存为 my_parametric_box.svg


In [8]:
import cv2
import numpy as np
from ultralytics import YOLO  # 导入 YOLO

def get_bottle_obb_yolo_from_camera(cap):
    """
    使用 YOLOv8 检测瓶子，并结合角点检测获取其最小外接矩形

    Args:
        cap (cv2.VideoCapture): VideoCapture 对象

    Returns:
        tuple: 瓶子的最小外接矩形，格式为 ((center_x, center_y), (width, height), angle)，
               如果未找到瓶子，则返回 None
    """
    # 1. 加载 YOLOv8 模型
    model = YOLO('F:/GitHub/Note/毕业设计思路/python_codes/runs/detect/train5/weights/best.pt')

    # 2. 从摄像头读取一帧图像
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        return None

    # 3. 使用 YOLOv8 进行目标检测
    results = model.predict(frame)

    # 4. 获取瓶子的检测结果
    bottle_rect = None
    for result in results:
        for box in result.boxes:
            if result.names[int(box.cls)] == 'bottle':  # 假设 'bottle' 是瓶子的类别名称
                xyxy = box.xyxy[0].cpu().numpy()  # 获取边界框坐标 (xmin, ymin, xmax, ymax)
                center_x = (xyxy[0] + xyxy[2]) / 2
                center_y = (xyxy[1] + xyxy[3]) / 2
                width = xyxy[2] - xyxy[0]
                height = xyxy[3] - xyxy[1]
                angle = 0  # YOLO 输出的是水平矩形

                # 5.  在 YOLO 检测到的区域内进行角点检测
                cropped_img = frame[int(xyxy[1]):int(xyxy[3]), int(xyxy[0]):int(xyxy[2])]
                cropped_gray = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)
                corners = cv2.goodFeaturesToTrack(cropped_gray, maxCorners=20, qualityLevel=0.01, minDistance=10)  # 获取角点

                if corners is not None and len(corners) > 4:
                    # 将角点转换为轮廓格式
                    corners_contour = np.array([corner.squeeze() for corner in corners], dtype=np.int32)
                    # 获取角点的最小外接矩形
                    bottle_rect = cv2.minAreaRect(corners_contour)
                else:
                    bottle_rect = ((center_x, center_y), (width, height), angle) # 否则用原始 YOLO 边界框

                break  # 找到一个瓶子就退出内层循环
        if bottle_rect:
            break  # 找到一个瓶子就退出外层循环
    return bottle_rect



# 1. 打开 USB 摄像头
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open camera")
    exit()

while True:
    # 2. 获取瓶子的最小外接矩形
    bottle_rect = get_bottle_obb_yolo_from_camera(cap)

    # 3. 绘制结果
    ret, frame = cap.read()  # 放在这里确保 frame 被读取
    if bottle_rect:
        box = cv2.boxPoints(bottle_rect)
        box = np.int0(box)
        cv2.drawContours(frame, [box], 0, (0, 0, 255), 2)
        cv2.imshow('Bottle Bounding Box (YOLO)', frame)
    else:
        cv2.imshow('Bottle Bounding Box (YOLO)', frame)  # 始终显示摄像头画面
        print('未找到瓶子')

    # 4. 按下 'q' 键退出
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 5. 释放摄像头
cap.release()
cv2.destroyAllWindows()



0: 480x640 1 bottle, 4.8ms
Speed: 1.4ms preprocess, 4.8ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bottle, 7.0ms
Speed: 0.8ms preprocess, 7.0ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bottle, 5.1ms
Speed: 0.7ms preprocess, 5.1ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bottle, 4.6ms
Speed: 0.7ms preprocess, 4.6ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bottle, 5.0ms
Speed: 0.8ms preprocess, 5.0ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bottle, 4.8ms
Speed: 0.7ms preprocess, 4.8ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bottle, 4.8ms
Speed: 0.8ms preprocess, 4.8ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bottle, 5.0ms
Speed: 0.7ms preprocess, 5.0ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x

In [10]:
import cv2
import numpy as np

def get_bottle_obb_cv(cap):
    """
    使用 OpenCV 从摄像头检测瓶子，并获取其最小外接矩形 (不使用 YOLO)

    Args:
        cap (cv2.VideoCapture): VideoCapture 对象

    Returns:
        tuple: 瓶子的最小外接矩形，格式为 ((center_x, center_y), (width, height), angle)，
               如果未找到瓶子，则返回 None
    """
    # 1. 从摄像头读取一帧图像
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        return None

    # 2. 转换为 HSV 色彩空间
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # 3. 定义瓶子的颜色范围 (根据实际情况调整)
    lower_green = np.array([30, 50, 50])
    upper_green = np.array([90, 255, 255])
    mask = cv2.inRange(hsv, lower_green, upper_green)

    # 4. 形态学操作去除噪点并连接断开的区域
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=2)  # 开运算
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2) # 闭运算

    # 5. 轮廓提取
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 6. 寻找最合适的轮廓
    bottle_rect = None
    if contours:
        # 6.1. 寻找面积最大的轮廓
        bottle_contour = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(bottle_contour)

        # 6.2. 面积过滤：移除过小的轮廓
        if area > 1000:  # 阈值需要根据实际情况调整
             bottle_rect = cv2.minAreaRect(bottle_contour)

    return bottle_rect



# 1. 打开 USB 摄像头
cap = cv2.VideoCapture(0)

# 设置分辨率
desired_width = 1920
desired_height = 1080
cap.set(cv2.CAP_PROP_FRAME_WIDTH, desired_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, desired_height)

if not cap.isOpened():
    print("Error: Could not open camera")
    exit()

while True:
    # 2. 获取瓶子的最小外接矩形
    bottle_rect = get_bottle_obb_cv(cap)

    # 3. 绘制结果
    ret, frame = cap.read()
    if bottle_rect:
        box = cv2.boxPoints(bottle_rect)
        box = np.int32(box)
        cv2.drawContours(frame, [box], 0, (0, 0, 255), 2)
        cv2.imshow('Bottle Bounding Box (CV)', frame)
    else:
        cv2.imshow('Bottle Bounding Box (CV)', frame) # 始终显示
        print('未找到瓶子')

    # 4. 按下 'q' 键退出
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 5. 释放摄像头
cap.release()
cv2.destroyAllWindows()


未找到瓶子
未找到瓶子
未找到瓶子
未找到瓶子
未找到瓶子
未找到瓶子
未找到瓶子
未找到瓶子
未找到瓶子
未找到瓶子
未找到瓶子
未找到瓶子
未找到瓶子
